In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import backend as K
%matplotlib inline
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Conv1D, MaxPooling2D, Flatten, Dense, Activation, Dropout
import os 
import cv2
import pickle
import sys
from pathlib import Path
import sklearn.model_selection as model_selection



In [0]:
DATA_BASE_PATH = Path('/content/drive/My Drive/Last_prueba')

image_paths = []
target_list = []

for image in list((DATA_BASE_PATH).glob('**/*.jpg')):
	
    image_class = image.parent.name
    image_path = str(image)
    
    if image_class.startswith('.'):
        print(image_class)
        continue
    
    target_list.append(image_class)
    image_paths.append(image_path)

In [0]:

paths_df = pd.DataFrame({'path': image_paths, 'class': target_list})

train_df, test_df = model_selection.train_test_split(paths_df, test_size=0.20)

IMG_SIZE = 300

datagen = ImageDataGenerator(rescale=1./255, 
                             rotation_range=20, 
                             zoom_range=0.15,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.15,
                             horizontal_flip=True,
                             fill_mode="nearest")

train_generator = datagen.flow_from_dataframe(dataframe=train_df,
                                              x_col='path', 
                                              y_col='class', 
                                              class_mode='categorical', 
                                              target_size=(IMG_SIZE, IMG_SIZE), 
                                              batch_size=64)

test_generator = datagen.flow_from_dataframe(dataframe=test_df,
                                              x_col='path', 
                                              y_col='class', 
                                              class_mode='categorical', 
                                              target_size=(IMG_SIZE, IMG_SIZE), 
                                              batch_size=64)

Found 3448 validated image filenames belonging to 7 classes.
Found 863 validated image filenames belonging to 7 classes.


In [0]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(300, 300, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(32, activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(units=7, activation='softmax'))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 73, 73, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 341056)            0         
__________

In [0]:
model.compile(
   loss='categorical_crossentropy',
   optimizer=keras.optimizers.Adadelta(),
   metrics=['accuracy'])

In [0]:
'''from keras.callbacks import ModelCheckpoint
filepath = ('/content/drive/My Drive/capturas_beers')
checkpointer = ModelCheckpoint(filepath=filepath'''

In [0]:
model.fit_generator(train_generator, epochs = 18, steps_per_epoch=len(train_generator), validation_data = test_generator, validation_steps = len(test_generator) )

Epoch 1/18
54/54 [==============================] - 223s 4s/step - loss: 2.2912 - acc: 0.2895 - val_loss: 1.3304 - val_acc: 0.6512
Epoch 2/18
54/54 [==============================] - 100s 2s/step - loss: 1.0790 - acc: 0.6064 - val_loss: 0.8980 - val_acc: 0.7462
Epoch 3/18
54/54 [==============================] - 107s 2s/step - loss: 0.7071 - acc: 0.7581 - val_loss: 0.4995 - val_acc: 0.9200
Epoch 4/18
54/54 [==============================] - 107s 2s/step - loss: 0.3410 - acc: 0.8899 - val_loss: 0.1795 - val_acc: 0.9676
Epoch 5/18
54/54 [==============================] - 107s 2s/step - loss: 0.2789 - acc: 0.9087 - val_loss: 0.1931 - val_acc: 0.9768
Epoch 6/18
54/54 [==============================] - 107s 2s/step - loss: 0.2032 - acc: 0.9343 - val_loss: 0.0947 - val_acc: 0.9815
Epoch 7/18
54/54 [==============================] - 107s 2s/step - loss: 0.1899 - acc: 0.9363 - val_loss: 0.0703 - val_acc: 0.9954
Epoch 8/18
54/54 [==============================] - 107s 2s/step - loss: 0.1518 - a

In [0]:
model.save('modelo_generator_final_7cats_nothing.h5')